<a href="https://colab.research.google.com/github/BouzCS/Sign-Language-Detection/blob/main/Sign_Language_Openpose%2BCNNLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import time
import numpy as np
from random import randint
import sys, os
from sys import platform
import pandas as pd
import argparse
from math import sqrt, acos, degrees, atan, degrees
import numpy as np
import random
import glob
from PIL import Image
import itertools
import _pickle as pickle
from datetime import datetime

In [ ]:
!pip install mediapipe yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.2 MB/s 
     |████████████████████████████████| 2.6 MB 61.4 MB/s 
     |████████████████████████████████| 2.0 MB 50.1 MB/s 
     |████████████████████████████████| 218 kB 72.7 MB/s 
     |████████████████████████████████| 112 kB 59.6 MB/s 
     |████████████████████████████████| 357 kB 66.1 MB/s 


In [ ]:
!git clone https://github.com/hafizas101/Deep-learning-based-human-action-recognition-using-human-skeletons.git

Cloning into 'Deep-learning-based-human-action-recognition-using-human-skeletons'...
remote: Enumerating objects: 95, done.
remote: Total 95 (delta 0), reused 0 (delta 0), pack-reused 95
Unpacking objects: 100% (95/95), done.


In [ ]:
!wget https://download.microsoft.com/download/3/c/a/3ca92c78-1c4a-4a91-a7ee-6980c1d242ec/MS-ASL.zip

--2022-06-07 11:43:03--  https://download.microsoft.com/download/3/c/a/3ca92c78-1c4a-4a91-a7ee-6980c1d242ec/MS-ASL.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.29.132.114, 2600:141b:f000:c08a::e59, 2600:141b:f000:c09f::e59
Connecting to download.microsoft.com (download.microsoft.com)|184.29.132.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2022053 (1.9M) [application/octet-stream]
Saving to: ‘MS-ASL.zip’

MS-ASL.zip          100%[===================>]   1.93M  --.-KB/s    in 0.05s   

2022-06-07 11:43:03 (39.4 MB/s) - ‘MS-ASL.zip’ saved [2022053/2022053]



In [ ]:
!unzip  /content/MS-ASL.zip

Archive:  /content/MS-ASL.zip
  inflating: MS-ASL/C-UDA-0.1_annotated_discussion.pdf  
  inflating: MS-ASL/MSASL_classes.json  
  inflating: MS-ASL/MSASL_synonym.json  
  inflating: MS-ASL/MSASL_test.json  
  inflating: MS-ASL/MSASL_train.json  
  inflating: MS-ASL/MSASL_val.json   
  inflating: MS-ASL/README.md        


In [ ]:
df = pd.read_json('/content/MS-ASL/MSASL_train.json')

In [ ]:
df

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,match [light-a-MATCH],match,0.000,0,0,0,83,match light-a-MATCH,830,360,30.000,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.057544618844985004, 0.21637457609176602, 1....",640,NaN
1,FAIL,fail,0.000,0,-1,0,74,FAIL,542,360,25.000,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.06577941775321901, 0.167171776294708, 0.939...",480,NaN
2,laugh,laugh,0.000,4,26,0,31,SignSchool Laugh with Legs 2,312,360,29.970,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.131885945796966, 0.32334136962890603, 1.0, ...",640,NaN
3,BOOK,book,0.000,0,-1,0,66,BOOK(3),38,360,25.000,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.055698871612548, 0.25173279643058705, 0.996...",480,NaN
4,sign-language,sign language,0.000,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360,29.970,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.039043992757797005, 0.24198183417320202, 1....",640,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16049,WRITE,write,539.939,32,17,16182,16301,ASL 1 Unit 1 Vocabulary,53,360,29.970,543.910,https://www.youtube.com/watch?v=fNg_sJ9f8EI,write,"[0.032177031040191005, 0.31613153219223, 1.0, ...",640,NaN
16050,hot,hot,0.000,2,42,0,127,Hot,135,360,29.657,4.282,https://www.youtube.com/watch?v=MkXUHhsMAns,hot,"[0.000367134809494, 0.20649650692939703, 1.0, ...",640,NaN
16051,hi,hi,0.000,0,0,0,47,Hi,379,360,29.970,1.568,www.youtube.com/watch?v=rhfJGeMDMzQ,hi,"[0.013705313205718002, 0.17314371466636602, 1....",640,NaN
16052,Learn,learn,0.000,135,-1,0,104,ASL Learn,22,360,30.000,3.467,https://www.youtube.com/watch?v=n-b2NMAwk28,learn,"[0.18269836902618403, 0.19255371391773202, 1.0...",480,NaN


In [ ]:
url_one_list = df['url'].tolist()
label_one_list = df['text'].tolist()


In [ ]:
# Calculate number of frames
cap = cv2. VideoCapture("video.mp4")
number_of_frames = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))

In [ ]:
from moviepy.editor import *
# Import everything needed to edit video clips
def trim_video(t1,t2):
  # loading video 
  clip = VideoFileClip("video.mp4")
  # getting only first seconds
  clip = clip.subclip(t1, t2)
  # showing clip
  clip.write_videofile("clip.mp4")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3579904/45929032 bytes (7.8%)3588096/45929032 bytes (7.8%)5857280/45929032 bytes (12.8%)5865472/45929032 bytes (12.8%)8323072/45929032 bytes (18.1%)10772480/45929032 bytes (23.5%)14106624/45929032 bytes (30.7%)16261120/45929032 bytes (35.4%)18964480/45929032 bytes (41.3%)21528576/45929032 bytes (46.9%)23429120/45929032 bytes (51.0%)25993216/45929032 bytes (56.6%)26001

In [ ]:
from google.colab import drive 
#drive.mount('/content/drive')

In [ ]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y, landmark_z])

    return landmark_point

In [ ]:
import cv2
import copy
import itertools

# Important : Preprocessing :
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [ ]:
# Important : Preprocessing :
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [ ]:
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
import tqdm

def get_keypoints(image_input):
  mp_drawing = mp.solutions.drawing_utils
  mp_drawing_styles = mp.solutions.drawing_styles
  mp_hands = mp.solutions.hands

  zero = np.zeros(63)
  col = range(1,127)
  df = pd.DataFrame(columns=col)
  try:
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.64) as hands:

        hands_list=[]
        data = []
        temp_data = []


        # Read an image, flip it around y-axis for correct handedness output (see
        # above).
        image = image_input
        image_width, image_height = image.shape[1], image.shape[0]

        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        try:
          for idx, hand_handedness in enumerate(results.multi_handedness):
            hands_list.append(hand_handedness.classification[0].label)
        except:
            pass
        hand_max_count = max(hands_list.count("Right"), hands_list.count("Left"))
        # Print handedness and draw hand landmarks on the image.
        #print('Handedness:', results.multi_handedness)
        annotated_image = image.copy()
        for hand_landmarks in results.multi_hand_landmarks:
          if results.multi_hand_landmarks:
              for i in range(21):
                data.append(min(int(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].x * image_width), image_width - 1))
                data.append(min(int(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].y * image_width), image_width - 1))
                data.append(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].z)
                temp_data.append(data[i*3:(i*3)+3])

        preproc_data=np.array(pre_process_landmark(temp_data))
          
        if "Right" in hands_list and "Left" in hands_list:
          df = df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))

        elif ("Right" in hands_list and "Left" not in hands_list):
          if hand_max_count==1:
            preproc_data = np.concatenate((preproc_data, zero), axis=None)

            df =df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))

          else:
            preproc_data = np.concatenate((preproc_data[:63], zero), axis=None)

            df =df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))       

        elif ("Right" not in hands_list and "Left" in hands_list):
          if hand_max_count==1:

            preproc_data = np.concatenate((zero, preproc_data), axis=None)

            df =df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))

          else:

            preproc_data = np.concatenate((zero, preproc_data[:63]), axis=None)

            df =df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))

  except:
    pass
    final=df.to_numpy()
    if len(final)==0 : final=0
  return final
            



In [ ]:
def save_file(file_name,data):
  np.save("/content/output_files/"+file_name+'.npy', data)


In [ ]:
global_data=[]

In [ ]:
!mkdir output_files

In [ ]:
!pip install psutil
import psutil # Can be found at https://pypi.org/project/psutil/

print('RAM memory % used:', psutil.virtual_memory()[2])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
RAM memory % used: 9.7


In [ ]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 5.1 MB/s 


In [ ]:
FRAMES_FIX = 30

from math import floor
import cv2
#from pytube import YouTube
from yt_dlp import YoutubeDL
import pandas as pd
import numpy as np
import mediapipe as mp
import tqdm
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import gc

columns = ["mediapipe_keypoint","box","label"]
RAM_LEVEL = psutil.virtual_memory()[2]
global_data = []

ydl_opts = {"format": "mp4[height=360]",
            'outtmpl': 'video.mp4',
            'ignoreerrors': True}


for i in tqdm.tqdm(range(0,1)):
  try:
    if RAM_LEVEL>60 or i == 34 or i==33 or i==51 or i==326:
      continue 
    #Always comment this 1st line then restart the code
    #Append to global dataframe
    #global_data.append([np.array(mediapipe_keypoints)[:30],df["box"][i],label])
    
    #Save an output every 7 videos (Solution for the memory problem)


    video_url = url_one_list[i]
    ## Choose a video stream with resolution of 360p
    #streams = YouTube(video_url).streams.filter(adaptive=True, subtype="mp4", resolution="360p", only_video=True)
    ## Check if there is a valid stream
    #if len(streams) == 0:
    #  raise "No suitable stream found for this YouTube video!"
    ## Download the video as video.mp4
    ##print("Downloading...")
    #streams[0].download(filename="video.mp4")
    ##print("Download completed.")

    with YoutubeDL(ydl_opts) as ydl:
      ydl.download([video_url])
    
    #print("Download completed.")
    trim_video(df["start_time"][i], df["end_time"][i])
    
    !rm video.mp4
    # Calculate number of frames
    cap = cv2. VideoCapture("clip.mp4")

    if RAM_LEVEL>60:
      continue 
    
    number_of_frames = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
    N = floor(number_of_frames/FRAMES_FIX)
    
    # The frame images will be stored in mediapipe_keypoints
    mediapipe_keypoints = []
    label=label_one_list[i]
    # Open the video file
    capture = cv2.VideoCapture('/content/clip.mp4')
    fps = capture.get(cv2.CAP_PROP_FPS)
    current_frame = 0
    while capture.isOpened():
      # Read the current frame
      ret, frame = capture.read()
      if RAM_LEVEL>60:
        continue 
      # Convert it to a PIL image (required for CLIP) and store it
      if ret == True:
        mediapipe_keypoints.append(get_keypoints(frame[:, :, ::-1]))
      else:
        break
      # Skip N frames
      if RAM_LEVEL>60:
        continue 
      current_frame += N
      
      capture.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
    
  except Exception as error:
          print('Caught this error: ' + repr(error))   




  0%|          | 0/1 [00:00<?, ?it/s]

[youtube] C37R_Ix8-qs: Downloading webpage
[youtube] C37R_Ix8-qs: Downloading android player API JSON
[youtube] C37R_Ix8-qs: Downloading MPD manifest
[youtube] C37R_Ix8-qs: Downloading MPD manifest
[info] C37R_Ix8-qs: Downloading 1 format(s): 18
[download] Destination: video.mp4
[download] 100% of 113.13KiB in 00:00               
[MoviePy] >>>> Building video clip.mp4
[MoviePy] Writing audio in clipTEMP_MPY_wvf_snd.mp3



100%|██████████| 62/62 [00:00<00:00, 1422.47it/s]

[MoviePy] Done.
[MoviePy] Writing video clip.mp4




100%|██████████| 84/84 [00:00<00:00, 166.60it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: clip.mp4 



100%|██████████| 1/1 [00:16<00:00, 16.07s/it]


In [ ]:
FRAMES_FIX = 30

from math import floor
import cv2
from pytube import YouTube
import pandas as pd
import numpy as np
import mediapipe as mp
import tqdm
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import gc

columns = ["mediapipe_keypoint","box","label"]
RAM_LEVEL = psutil.virtual_memory()[2]
global_data = np.empty(0)




for i in tqdm.tqdm(range(len(df))):
  try:
    if RAM_LEVEL>60:
      continue 
    #Always comment this 1st line then restart the code
    #Append to global dataframe
    global_data=np.append(global_data,np.array([(mediapipe_keypoints)[:30],df["box"][i],label]))
    
    #Save an output every 50 videos (Solution for the memory problem)
    if (i%50 == 0):
        save_file("output_"+str(i//10),global_data)
        print('\n'+'######################################## File Created ########################################')
        del global_data
        gc.collect()
        global_data=np.empty(0)

    video_url = url_one_list[i]
    ## Choose a video stream with resolution of 360p
    #streams = YouTube(video_url).streams.filter(adaptive=True, subtype="mp4", resolution="360p", only_video=True)
    ## Check if there is a valid stream
    #if len(streams) == 0:
    #  raise "No suitable stream found for this YouTube video!"
    ## Download the video as video.mp4
    ##print("Downloading...")
    #streams[0].download(filename="video.mp4")
    ##print("Download completed.")

    with YoutubeDL(ydl_opts) as ydl:
      ydl.download([video_url])
    
    #print("Download completed.")
    trim_video(df["start_time"][i], df["end_time"][i])
    
    !rm video.mp4
    # Calculate number of frames
    cap = cv2. VideoCapture("clip.mp4")

    if RAM_LEVEL>60:
      continue 
    
    number_of_frames = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
    N = floor(number_of_frames/FRAMES_FIX)
    
    # The frame images will be stored in mediapipe_keypoints
    mediapipe_keypoints = []
    label=label_one_list[i]
    # Open the video file
    capture = cv2.VideoCapture('/content/clip.mp4')
    fps = capture.get(cv2.CAP_PROP_FPS)
    current_frame = 0
    while capture.isOpened():
      # Read the current frame
      ret, frame = capture.read()
      if RAM_LEVEL>60:
        continue 
      # Convert it to a PIL image (required for CLIP) and store it
      if ret == True:
        if len(mediapipe_keypoints) > 180:
          break
        mediapipe_keypoints.append(get_keypoints(frame[:, :, ::-1]))
      else:
        break
      # Skip N frames
      if RAM_LEVEL>60:
        continue 
      current_frame += N
      
      capture.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
    
  except Exception as error:
          print('Caught this error: ' + repr(error))



  0%|          | 0/16054 [00:00<?, ?it/s]


######################################## File Created ########################################
[youtube] C37R_Ix8-qs: Downloading webpage
[youtube] C37R_Ix8-qs: Downloading android player API JSON
[youtube] C37R_Ix8-qs: Downloading MPD manifest
[youtube] C37R_Ix8-qs: Downloading MPD manifest
[info] C37R_Ix8-qs: Downloading 1 format(s): 18
[download] Destination: video.mp4
[download] 100% of 113.13KiB in 00:00               
[MoviePy] >>>> Building video clip.mp4
[MoviePy] Writing audio in clipTEMP_MPY_wvf_snd.mp3



100%|██████████| 62/62 [00:00<00:00, 1530.23it/s]

[MoviePy] Done.
[MoviePy] Writing video clip.mp4




100%|██████████| 84/84 [00:01<00:00, 64.75it/s] 


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: clip.mp4 



  0%|          | 5/16054 [00:20<19:47:39,  4.44s/it]

Caught this error: UnboundLocalError("local variable 'final' referenced before assignment")
[youtube] 1AyT77LqJzQ: Downloading webpage
[youtube] 1AyT77LqJzQ: Downloading android player API JSON


ERROR: [youtube] 1AyT77LqJzQ: Video unavailable
  0%|          | 6/16054 [00:21<14:19:44,  3.21s/it]

Caught this error: OSError('MoviePy error: the file video.mp4 could not be found!\nPlease check that you entered the correct path.')
[youtube] 1AyT77LqJzQ: Downloading webpage


In [ ]:
import pandas as pd 
pd.DataFrame(np.array([np.array(mediapipe_keypoints)[:30],df["box"][i],label])).to_csv("file.csv")

In [ ]:
global_data

array([array([list([]), list([]), list([]), list([]), list([]), list([]),
              list([]), list([]), list([]), list([]), list([]), list([]),
              list([[0.0, 0.0, -1.3192068256497996e-09, -0.06578947368421052, -0.35526315789473684, 6.285747268090123e-05, -0.15789473684210525, -0.6710526315789473, 5.472698371465269e-05, -0.2631578947368421, -0.8421052631578947, 2.2872871915368658e-05, -0.3684210526315789, -0.9210526315789473, -2.431806397477263e-05, -0.11842105263157894, -0.9342105263157895, 5.2967133294594915e-05, -0.34210526315789475, -1.0, -1.641015344495444e-05, -0.4868421052631579, -1.0, -9.677155359991287e-05, -0.5789473684210527, -1.0, -0.00014449849626735638, -0.14473684210526316, -0.868421052631579, -1.1662934220543033e-05, -0.3684210526315789, -0.9342105263157895, -3.062922041863203e-05, -0.5, -0.9342105263157895, -6.942830881790112e-05, -0.6052631578947368, -0.9210526315789473, -0.00010573627476237322, -0.18421052631578946, -0.7763157894736842, -7.463236167830

In [ ]:
#np.set_printoptions(threshold=sys.maxsize)
np.array(mediapipe_keypoints)[:30].shape

(30,)

In [ ]:
np.save('test3.npy', np.array(mediapipe_keypoints)[:30])

In [ ]:
x = np.load('test3.npy',allow_pickle=True)


In [ ]:
x

array([array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          

In [ ]:
!zip -r /content/0-500.zip /content/output_files

  adding: content/output_files/ (stored 0%)
  adding: content/output_files/output_1580.txt (deflated 91%)
  adding: content/output_files/output_0.txt (stored 0%)
  adding: content/output_files/output_1575.txt (deflated 93%)
  adding: content/output_files/output_1585.txt (deflated 97%)


In [ ]:
#Original
FRAMES_FIX = 30

from math import floor
import cv2
from pytube import YouTube
import pandas as pd
import numpy as np
import mediapipe as mp
import tqdm
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

columns = ["mediapipe_keypoint","box","label"]

global_data = pd.DataFrame(columns=columns)


for i in tqdm.tqdm(range(len(df))):
  try:
    
    #Append to global dataframe
    global_data = global_data.append(pd.DataFrame([[np.array(mediapipe_keypoints)[:30],df["box"][i],label]],columns=columns))
    
    with YoutubeDL(ydl_opts) as ydl:
      ydl.download([video_url])
    
    #print("Download completed.")
    trim_video(df["start_time"][i], df["end_time"][i])
    
    !rm video.mp4
    # Calculate number of frames
    cap = cv2. VideoCapture("clip.mp4")
    number_of_frames = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
    N = floor(number_of_frames/FRAMES_FIX)
    
    # The frame images will be stored in mediapipe_keypoints
    mediapipe_keypoints = []
    label=label_one_list[i]
    # Open the video file
    capture = cv2.VideoCapture('/content/clip.mp4')
    fps = capture.get(cv2.CAP_PROP_FPS)
    current_frame = 0
    while capture.isOpened():
      # Read the current frame
      ret, frame = capture.read()
      # Convert it to a PIL image (required for CLIP) and store it
      if ret == True:
        if len(mediapipe_keypoints) > 180:
          break
        mediapipe_keypoints.append(frame[:, :, ::-1])
      else:
        break
      # Skip N frames
      current_frame += N
      
      capture.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
    
  except Exception as error:
          print('Caught this error: ' + repr(error))   




  0%|          | 0/16054 [00:00<?, ?it/s]

[youtube] s-QgMaUgtTQ: Downloading webpage
[youtube] s-QgMaUgtTQ: Downloading android player API JSON
[info] s-QgMaUgtTQ: Downloading 1 format(s): 18
[download] Destination: video.mp4
[download] 100% of 591.03KiB in 00:00               
[MoviePy] >>>> Building video clip.mp4
[MoviePy] Writing audio in clipTEMP_MPY_wvf_snd.mp3



100%|██████████| 62/62 [00:00<00:00, 1236.55it/s]

[MoviePy] Done.
[MoviePy] Writing video clip.mp4




100%|██████████| 84/84 [00:00<00:00, 191.50it/s]


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: clip.mp4 



  0%|          | 30/16054 [01:37<15:09:54,  3.41s/it]

[youtube] s-QgMaUgtTQ: Downloading webpage
[youtube] s-QgMaUgtTQ: Downloading android player API JSON


  0%|          | 30/16054 [01:37<14:29:49,  3.26s/it]


KeyboardInterrupt: ignored

## Select a YouTube video

Paste a link to a YouTube video or choose one of the examples. 

Choose a value for `N` which defines how many frames should be skipped. `N = 30` is usually about 1 frame every second.

In [ ]:
# Dashcam video of driving around San Francisco
video_url = "https://www.youtube.com/watch?v=C37R_Ix8-qs"  

# How much frames to skip
N = 6

FRAMES_FIX = 30

from math import floor
N = floor(number_of_frames/FRAMES_FIX)
N

0

## Setup the environment

Install CLIP and install its dependencies.

In [ ]:
# pytube is used to download videos from YouTube
!pip install pytube

# TQDM
!pip install tqdm
# Intall a newer version of plotly
!pip install plotly==4.14.3

# Install CLIP from the GitHub repo
!pip install git+https://github.com/openai/CLIP.git

# Install torch 1.7.1 with GPU support
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html

## Download the video and process with CLIP

In this section the video is downloaded, every N-th frame is extracted and each frame is processed using CLIP.

Download the video locallyat 360p resoultion using `pytube`. A bigger resolution is not needed because CLIP scales down the images anyway.

In [ ]:
image = cv2.imread("/content/download.png")
image_width, image_height = image.shape[1], image.shape[0]
image_width, image_height

AttributeError: ignored

In [ ]:
from pytube import YouTube

# Choose a video stream with resolution of 360p
streams = YouTube(video_url).streams.filter(adaptive=True, subtype="mp4", resolution="360p", only_video=True)

# Check if there is a valid stream
if len(streams) == 0:
  raise "No suitable stream found for this YouTube video!"

# Download the video as video.mp4
print("Downloading...")
streams[0].download(filename="video.mp4")
print("Download completed.")

RegexMatchError: ignored

In [ ]:
# Dashcam video of driving around San Francisco
video_url = "https://www.youtube.com/watch?v=C37R_Ix8-qs"  

# How much frames to skip
N = 6

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
ffmpeg_extract_subclip("video.mp4", round(df["start_time"][i]), round(df["end_time"][i]), targetname="final_video.mp4")

Extract every `N-th` frame of the video.

In [ ]:
import cv2
from PIL import Image

# The frame images will be stored in video_frames
video_frames = []

# Open the video file
capture = cv2.VideoCapture('/content/clip.mp4')
fps = capture.get(cv2.CAP_PROP_FPS)

current_frame = 0
while capture.isOpened():
  # Read the current frame
  ret, frame = capture.read()

  # Convert it to a PIL image (required for CLIP) and store it
  if ret == True:
    video_frames.append(Image.fromarray(frame[:, :, ::-1]))
    print(frame.shape)
  else:
    break

  # Skip N frames
  current_frame += 6
  capture.set(cv2.CAP_PROP_POS_FRAMES, current_frame)

# Print some statistics
print(f"Frames extracted: {len(video_frames)}")

In [ ]:
Image.fromarray(video_frames[25])

In [ ]:

numpy_array = get_keypoints(video_frames[13])

numpy_array

Load the public CLIP model.

In [ ]:
import clip
import torch

# Load the open CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

Encode all frames using CLIP. The encoding is done in batches for a better efficiency.

In [ ]:
import math
import numpy as np
import torch

# You can try tuning the batch size for very large videos, but it should usually be OK
batch_size = 256
batches = math.ceil(len(video_frames) / batch_size)

# The encoded features will bs stored in video_features
video_features = torch.empty([0, 512], dtype=torch.float16).to(device)

# Process each batch
for i in range(batches):
  print(f"Processing batch {i+1}/{batches}")

  # Get the relevant frames
  batch_frames = video_frames[i*batch_size : (i+1)*batch_size]
  
  # Preprocess the images for the batch
  batch_preprocessed = torch.stack([preprocess(frame) for frame in batch_frames]).to(device)
  
  # Encode with CLIP and normalize
  with torch.no_grad():
    batch_features = model.encode_image(batch_preprocessed)
    batch_features /= batch_features.norm(dim=-1, keepdim=True)

  # Append the batch to the list containing all features
  video_features = torch.cat((video_features, batch_features))

# Print some stats
print(f"Features: {video_features.shape}")

In [ ]:
import os
from os.path import exists, join, basename, splitext
# initiating variable for cloning openpose
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
from IPython.display import YouTubeVideo

In [ ]:
YOUTUBE_ID = '0daS_SDCT_U'
YouTubeVideo(YOUTUBE_ID)
import io
import base64
from IPython.display import HTML
file_name="output.mp4"
width=640
height=480
video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))